In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from datetime import timedelta
#import datetime
%matplotlib inline

In [2]:
df = pd.read_csv(r"C:\Users\Janwe\OneDrive\Desktop\Bachelor_Arbeit\Daten\Electirc Vehicles\REM2030_v2015.csv", sep=";")
car_info = pd.read_csv(r"C:\Users\Janwe\OneDrive\Desktop\Bachelor_Arbeit\Daten\Electirc Vehicles\REM2030_v2015_car_info.csv",sep=";")

In [3]:
df_tmp = df[["deptyear","deptmonth","deptday","depthour","deptminute","arryear","arrmonth","arrday","arrhour","arrminute"]]

In [4]:
df_tmp.astype("int32").dtypes

deptyear      int32
deptmonth     int32
deptday       int32
depthour      int32
deptminute    int32
arryear       int32
arrmonth      int32
arrday        int32
arrhour       int32
arrminute     int32
dtype: object

In [5]:
electric_cars = pd.DataFrame(columns=["id","deptTime","arrTime"])
electric_cars[["id","distance", "distance_to_company"]] = df[["id","distance", "distance_to_company"]].copy()

In [6]:
deptTime = []
for i in range(len(df)):
    l = df_tmp.loc[i]
    deptTime.append(datetime(l[0],l[1],l[2],l[3],l[4]))

arrTime = []
for i in range(len(df)):
    l = df_tmp.loc[i]
    arrTime.append(datetime(l[5],l[6],l[7],l[8],l[9]))

In [7]:
electric_cars["deptTime"] = pd.Series(deptTime)
electric_cars["arrTime"] = pd.Series(arrTime)

transporter = car_info[car_info["vehicle_size"]=="transporter"]
special_vehicle = car_info[car_info["vehicle_size"]=="special vehicle"]

In [8]:
#exclude transporter and special vehicles
exclude = pd.concat([transporter["id"],special_vehicle["id"]])
ev_excluded = electric_cars[~electric_cars.id.isin(exclude)]
car_info_excluded = car_info[~car_info.id.isin(exclude)]
car_info_excluded = car_info_excluded[['id','vehicle_size','federal_state','company_id']]

In [9]:
car_info_excluded.head()

,id,vehicle_size,federal_state,company_id
0,1106000161,small,BW,110615
1,1106000171,large,BW,110615
2,1106000181,small,BW,110615
3,1106000191,medium,BW,110615
4,1106000201,small,BW,110615


In [10]:
ev = ev_excluded.copy()
ev["deptTime_round"] = pd.to_datetime(ev["deptTime"].dt.floor("15min"))
ev["arrTime_round"] = pd.to_datetime(ev["arrTime"].dt.floor('15min'))
ev = ev.drop(columns="distance_to_company")
ev["week"] = ev["deptTime"].dt.week
ev["oneWeek"] = ev.id.astype(str) + ev["week"].astype(str)

C:\Users\Janwe\anaconda3\envs\condaPython3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [11]:
# Index Levels
outside = ev["oneWeek"]
inside = ev.index
hier_index = list(zip(outside,inside))
hier_index = pd.MultiIndex.from_tuples(hier_index)

In [12]:
hier_ev = ev.set_index(hier_index)
hier_ev.index.names = ["week",None]
hier_ev.drop(columns=["week","oneWeek"],inplace=True)

In [13]:
hier_ev

id            deptTime             arrTime  \
week                                                                     
110600016124 0      1106000161 2011-06-16 08:07:00 2011-06-16 09:55:00   
             1      1106000161 2011-06-16 10:32:00 2011-06-16 10:34:00   
             2      1106000161 2011-06-16 15:15:00 2011-06-16 15:17:00   
             3      1106000161 2011-06-17 11:47:00 2011-06-17 11:51:00   
             4      1106000161 2011-06-17 11:51:00 2011-06-17 11:52:00   
...                        ...                 ...                 ...   
15016552587  86795  1501655258 2015-02-12 10:48:00 2015-02-12 11:02:00   
             86796  1501655258 2015-02-12 11:07:00 2015-02-12 11:14:00   
             86797  1501655258 2015-02-12 16:55:00 2015-02-12 17:12:00   
             86798  1501655258 2015-02-12 17:55:00 2015-02-12 18:55:00   
             86799  1501655258 2015-02-12 21:52:00 2015-02-12 21:53:00   

                    distance      deptTime_round       arrTime_round  
week                                                                  
110600016124 0          2.66 2011-06-16 08:00:00 2011-06-16 09:45:00  
             1          3.19 2011-06-16 10:30:00 2011-06-16 10:30:00  
             2          0.63 2011-06-16 15:15:00 2011-06-16 15:15:00  
             3          1.70 2011-06-17 11:45:00 2011-06-17 11:45:00  
             4          1.54 2011-06-17 11:45:00 2011-06-17 11:45:00  
...                      ...                 ...                 ...  
15016552587  86795      0.50 2015-02-12 10:45:00 2015-02-12 11:00:00  
             86796      0.40 2015-02-12 11:00:00 2015-02-12 11:00:00  
             86797      1.60 2015-02-12 16:45:00 2015-02-12 17:00:00  
             86798     44.70 2015-02-12 17:45:00 2015-02-12 18:45:00  
             86799      0.20 2015-02-12 21:45:00 2015-02-12 21:45:00  

[65006 rows x 6 columns]

In [14]:
index = list(hier_ev.index)
weeks = []

for i in range(len(index)):
    weeks.append(index[i][0])
    

In [15]:
trips = hier_ev.drop(["id","arrTime","deptTime"],axis=1)
trips["deptTime"] = trips["deptTime_round"].dt.strftime("%w - %H:%M")
trips["arrTime"] = trips["arrTime_round"].dt.strftime("%w - %H:%M")
trips.drop(["arrTime_round","deptTime_round"],axis=1,inplace=True)

In [16]:
trips.loc[["110600016125"]]

distance   deptTime    arrTime
week                                           
110600016125 6       3.17  1 - 10:00  1 - 10:30
             7      18.14  1 - 13:15  1 - 13:30
             8       2.65  2 - 09:30  2 - 09:45
             9       9.88  2 - 10:45  2 - 11:15
             10      2.84  2 - 14:45  2 - 15:00
             11      2.30  5 - 07:30  5 - 07:30
             12      5.58  5 - 07:30  5 - 07:45
             13      7.11  5 - 08:00  5 - 08:15
             14      5.93  5 - 08:30  5 - 08:45
             15      1.20  5 - 09:30  5 - 09:30
             16      1.46  5 - 09:45  5 - 09:45

In [17]:
week_trips = pd.unique(weeks)

In [18]:
def timeDiff(dept,arr):
    FMT = '%H:%M'
    tdelta = datetime.strptime(arr, FMT) - datetime.strptime(dept, FMT)
    if tdelta.days < 0:
        tdelta = timedelta(days=0,
                    seconds=tdelta.seconds, microseconds=tdelta.microseconds)
    return tdelta

In [19]:
from datetime import time
start = datetime(2020,10,5,0,0)
result = []
for a in range(672):
    start += timedelta(minutes=15)
    result.append(start.time())
    

In [20]:
Av = pd.DataFrame(data= pd.Series(result), columns=["time"])

In [21]:
a = []
for i in range(len(Av)):
    a.append(1)
    
Av["a"] = a

In [22]:
weekday = []
day = 0 
for i in range(len(Av)):
    if(Av["time"][i]== Av["time"][671]):
        day = day + 1
    weekday.append(day)
    
Av["weekday"] = weekday

In [23]:
info = pd.DataFrame(columns=["A","distance"])
dist = []
for l in range(len(Av)):
    dist.append(0.0)
    
info["A"] = Av["a"].copy()
info["distance"] = dist


In [24]:
frames = {}

for k in week_trips[:1750]:
    frames[k] = info.copy()


In [25]:
for i in range(len(Av)):
    Av["time"][i] = Av["time"][i].strftime("%H:%M")

C:\Users\Janwe\anaconda3\envs\condaPython3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#dict 

def sepFunc(d2):
    go = False
    for k in d2.keys():
        t = trips.loc[[k]].droplevel(level=0)
        t.reset_index(inplace=True,drop=True)
        for j in range(len(t)):
            for i in range(len(Av)):
                time = Av["time"][i]
                if(int(t.iloc[j]["deptTime"][0]) == Av["weekday"][i]):
                    if(t.iloc[j]["deptTime"][4:] == time):
                        if(go == False):
                            d2.get(k)["distance"][i-1] = t.iloc[j]["distance"]
                        go = True
                    if(go):
                        if(t.iloc[j]["arrTime"][4:] == time):
                            d2.get(k)["A"][i] =  0
                            go = False
                        if(t.iloc[j]["arrTime"][4:] is not time):
                            d2.get(k)["A"][i] =  0
                            
    return d2

In [ ]:
tmp = sepFunc(frames)
out  = list(tmp.values())
df_out = pd.concat(out)

In [ ]:
df_out.to_csv(r"C:\Users\Janwe\OneDrive\Desktop\Bachelor_Arbeit\Daten\ev_Av_consumption_1750.csv",index=False)

In [ ]:
together = []
for i in range(0,len(df_out),672):
    together.append(df_out[i:i+672])
    

In [ ]:
df_ev = pd.read_csv(r"C:\Users\Janwe\OneDrive\Desktop\Bachelor_Arbeit\Daten\ev_Av_consumption_1750.csv")
df = pd.read_csv(r"C:/Users/Janwe/OneDrive/Desktop/Bachelor_Arbeit/Daten/base_data_2019_65000H.csv")

In [ ]:
#splitting the data for the EVs into two lists

ev_list = []
for i in range(0,len(df_ev),672):
    ev_list.append(df_ev[i:i+672].reset_index(drop=True))

Av_list = []
for i in ev_list:
    Av_list.append(i["A"])
    
consumption_list = []

for i in ev_list:
    consumption_list.append((i["distance"]*0.00015)/4)

## 1st & 2nd Quarter

The second quarter has the same length as the first, thus the first can be used

In [1]:
len1st = len(df["date"][:8640])

NameError: name 'df' is not defined

In [ ]:
#2nd quarter
df["date"][8640:17280]

In [ ]:
consumption_1stQuarter = consumption_list.copy()

for j in range(11):
    for i in range(len(consumption_list)):
        consumption_1stQuarter[i] = consumption_1stQuarter[i].append(consumption_list[i])
        

for i in range(len(consumption_list)):        
    consumption_1stQuarter[i] = consumption_1stQuarter[i].append(consumption_list[i][:576])
    consumption_1stQuarter[i].reset_index(inplace = True ,drop=True)
    
for i in range(len(consumption_1stQuarter)):
    for j in range(len1st-6, len1st):
        consumption_1stQuarter[i][len1st-6:][j]=0

In [ ]:
Av_1stQuarter = Av_list.copy()

In [ ]:
for j in range(11):
    for i in range(len(Av_list)):
        Av_1stQuarter[i] = Av_1stQuarter[i].append(Av_list[i]).reset_index(drop=True)
        

for i in range(len(Av_list)):        
    Av_1stQuarter[i] = Av_1stQuarter[i].append(Av_list[i][:576])
    Av_1stQuarter[i].reset_index(inplace=True,drop=True)
    
for i in range(len(Av_1stQuarter)):
    for j in range(len1st-6, len1st-6):
        Av_1stQuarter[i][len1st-6:][j]=1

In [ ]:
df_1stAnd2ndQuarter = pd.DataFrame(columns=["date","Av", "consumption"])
df_1stAnd2ndQuarter["date"] = df["date"][:8640]
df_1stAnd2ndQuarter["Av"] = Av_1stQuarter
df_1stAnd2ndQuarter["consumption"] = consumption_1stQuarter

In [ ]:
8640/4/24/7

In [ ]:
len(Av_1stQuarter)

## 3rd Quarter

In [ ]:
len3rd = len(df["date"][17280:26112])

In [ ]:
consumption_3rdQuarter = consumption_list.copy()

for j in range(12):
    for i in range(len(consumption_list)):
        consumption_3rdQuarter[i] = consumption_3rdQuarter[i].append(consumption_list[i])
        
for i in range(len(consumption_list)):        
    consumption_3rdQuarter[i] = consumption_3rdQuarter[i].append(consumption_list[i][:96])
    consumption_3rdQuarter[i].reset_index(inplace = True ,drop=True)

for i in range(len(consumption_3rdQuarter)):
    for j in range(len3rd-6,len3rd):
        consumption_3rdQuarter[i][len(consumption_3rdQuarter[1])-6:][j]=0

In [ ]:
Av_3rdQuarter = Av_list.copy()

for j in range(12):
    for i in range(len(Av_list)):
        Av_3rdQuarter[i] = Av_3rdQuarter[i].append(Av_list[i]).reset_index(drop=True)
        

for i in range(len(Av_list)):        
    Av_3rdQuarter[i] = Av_3rdQuarter[i].append(Av_list[i][:96])
    Av_3rdQuarter[i].reset_index(inplace=True,drop=True)
    
for i in range(len(Av_3rdQuarter)):
    for j in range(len(Av_3rdQuarter[1])-6,len(Av_3rdQuarter[1])):
        Av_3rdQuarter[i][len(Av_3rdQuarter[1])-6:][j]=1

## 4th Quarter

In [ ]:
len4th = len(df["date"][26112:35040])

In [ ]:
len4th

In [ ]:
consumption_4thQuarter = consumption_list.copy()

for j in range(12):
    for i in range(len(consumption_list)):
        consumption_4thQuarter[i] = consumption_4thQuarter[i].append(consumption_list[i])
        
for i in range(len(consumption_list)):        
    consumption_4thQuarter[i] = consumption_4thQuarter[i].append(consumption_list[i][:192])
    consumption_4thQuarter[i].reset_index(inplace = True ,drop=True)

for i in range(len(consumption_4thQuarter)):
    for j in range(len4th-6,len4th):
        consumption_4thQuarter[i][len4th-6:][j]=0

In [ ]:
Av_4thQuarter = Av_list.copy()

for j in range(12):
    for i in range(len(Av_list)):
        Av_4thQuarter[i] = Av_4thQuarter[i].append(Av_list[i]).reset_index(drop=True)
        

for i in range(len(Av_list)):        
    Av_4thQuarter[i] = Av_4thQuarter[i].append(Av_list[i][:192])
    Av_4thQuarter[i].reset_index(inplace=True,drop=True)
    
for i in range(len(Av_4thQuarter)):
    for j in range(len(Av_4thQuarter[1])-6,len(Av_4thQuarter[1])):
        Av_4thQuarter[i][len(Av_4thQuarter[1])-6:][j]=1